In [86]:
%load_ext autoreload
%autoreload 2
from DeepSeqG9 import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
hgmd = "/Users/jiayao/Work/DeepSeq/dat/hgmd/hgmd.std.csv"
cancer = "/Users/jiayao/Work/DeepSeq/dat/hotspot/cancer_hotspot.std.csv"
clinvar = "/Users/jiayao/Work/DeepSeq/dat/clinvar/clinvar.std.csv"
pr_coding_genes = "/Users/jiayao/Work/Resources/protein-coding_gene.txt"

In [ ]:
ins.mergeTable(pr_coding_genes,  hgmd, clinvar, cancer)

In [ ]:
gnomad = ""
ins.MakeBenighVars(gnomad, "hgmd_clinvar_cancer_combined_pathogenic_variants.tsv")

In [ ]:
df = pd.read_csv(hgmd, sep = "\t")
print df.columns.values

In [ ]:
dat = []
genes = []
for k,v in res.items():
    #print k, len(v)
    genes.append(k)
    dat.append(len(v))
genes.sort()

In [ ]:
mpl.pyplot.hist(dat, bins = 200)
plt.xlim(0,200)
plt.show()
print len([x for x in dat if x >10])
print len([x for x in dat if x >20])

In [ ]:
ins.mergeTable(pr_coding_genes, hgmd, clinvar, cancer)

In [ ]:
gnomad_file = "/Users/jiayao/Work/DeepSeq/dat/gnomAD/test.vcf.gz"
gnomad_file = "/Users/jiayao/Work/DeepSeq/dat/gnomAD/haha.vcf.gz"

In [ ]:
head = ['genename', 'assembly', 'chr', 'pos', 'ref', 'alt', 'ref_aa', 'alt_aa', 'swissprot_pos', 'ensp', 'swissprot', 
        'dbsnp', 'source', 'source_id', 'target', 'descr', 'var_id', 'swissprot_var_id', 'af']
print zip(range(19), head)

CSQ_FMT = 'Allele|Consequence|IMPACT|SYMBOL|Gene|Feature_type|Feature|BIOTYPE|EXON|INTRON|HGVSc|HGVSp|cDNA_position|CDS_position|Protein_position|Amino_acids|Codons|Existing_variation|ALLELE_NUM|DISTANCE|STRAND|FLAGS|VARIANT_CLASS|MINIMISED|SYMBOL_SOURCE|HGNC_ID|CANONICAL|TSL|APPRIS|CCDS|ENSP|SWISSPROT|TREMBL|UNIPARC|GENE_PHENO|SIFT|PolyPhen|DOMAINS|HGVS_OFFSET|GMAF|AFR_MAF|AMR_MAF|EAS_MAF|EUR_MAF|SAS_MAF|AA_MAF|EA_MAF|ExAC_MAF|ExAC_Adj_MAF|ExAC_AFR_MAF|ExAC_AMR_MAF|ExAC_EAS_MAF|ExAC_FIN_MAF|ExAC_NFE_MAF|ExAC_OTH_MAF|ExAC_SAS_MAF|CLIN_SIG|SOMATIC|PHENO|PUBMED|MOTIF_NAME|MOTIF_POS|HIGH_INF_POS|MOTIF_SCORE_CHANGE|LoF|LoF_filter|LoF_flags|LoF_info'
CSQ_FMT = CSQ_FMT.split("|")
print CSQ_FMT
def getINFO(info_string):
    infolist = info_string.split(';')
    infodict = {}
    for kv in infolist:
        kv = kv.split('=')
        if len(kv) == 2:
            k, v = kv
            if k in infodict:
                infodict[k].append(v) 
            else:
                infodict[k] = [v]
    return infodict
def searchCANONICAL(CSQLIST):
    for csq in CSQLIST:
        csq = csq.split("|")
        CSQ = dict(zip(CSQ_FMT, csq))
        if CSQ["CANONICAL"] == "YES":
            #return csq
            return "YES"
    return None
def displayCSQ(head, _dict):
    for item in head:
        print "{}:{}\t".format(item, _dict[item]),

In [ ]:
hand = gzip.open(gnomad_file, 'rb')
outfil = csv.writer(open("benigh.tsv", 'wb'), delimiter="\t")
outfil.writerow(head)
for l in hand:
    if l.startswith("#"):
        continue
    llist = l.strip().split("\t")
    Chr, Pos, ID, ref, alt = llist[0:5]
    if len(ref) != 1:
        continue
    INFO_string = llist[7]
    info_dict = getINFO(INFO_string)
    #print info_dict
    #CSQ = searchCANONICAL(info_dict["CSQ"])
    CSQ = info_dict["CSQ"][0].split("|")
    CSQ = dict(zip(CSQ_FMT, CSQ))
    if CSQ["Consequence"] != "missense_variant":
        continue
    AA = CSQ["Amino_acids"].split("/")
    res = [""]*19
    res[0] = info_dict["Gene.refGene"][0]
    res[1] = 'b37'
    res[2] = Chr
    res[3] = Pos
    res[4] = ref
    res[5] = alt.split(",")[0]
    res[6] = AA[0]
    res[7] = AA[1]
    res[8] = CSQ["Protein_position"]
    res[9] = CSQ["ENSP"]
    res[10] = CSQ["SWISSPROT"]
    res[11] = info_dict["avsnp147"][0]
    res[12] = "gnomAD_exome"
    res[13] = "-"
    res[14] = "-"
    res[15] = "-"
    res[16] = "{}_{}_{}_{}".format(Chr,Pos,ref,res[5])
    res[17] = "{}_{}_{}_{}".format(res[10],res[8],res[6],res[7])
    res[18] = info_dict["gnomAD_exome_ALL"][0]
    outfil.writerow(res)

In [ ]:
f1 = "/Users/jiayao/Work/DeepSeq/src/benigh.tsv"
f2 = "/Users/jiayao/Work/DeepSeq/src/hgmd_clinvar_cancer_combined_pathogenic_variants.tsv"

In [ ]:
reader1 = csv.reader(open(f2, 'rb'), delimiter="\t")
header = reader1.next()
pathvarset = []
for row in reader1:
    pathvarset.append(row[16])
pathvarset = set(pathvarset)

In [ ]:
reader2 = csv.reader(open(f1, 'rb'), delimiter="\t")
header = reader2.next()
outfil = csv.writer(open("benigh.clean.tsv", 'wb'), delimiter="\t")
outfil.writerow(head)
for row in reader2:
    varid = row[16]
    if varid in pathvarset:
        continue
    outfil.writerow(row)


# Count

In [ ]:
Gene2PDB = {}
for l in open("UniProt_PDB_ID_List.txt", 'rb'):
    if l.startswith("Entry"):
        continue
    llist = l.strip().split()
    if len(llist) < 3: #No PDB file
        continue
    if not llist[2].endswith(";"):
        continue
    Uniprot = llist[0]
    gene = llist[1]
    pdb = llist[2].split(";")[0]
    Gene2PDB[Uniprot] = pdb

In [ ]:
benigh = "/Users/jiayao/Work/DeepSeq/src/benigh.clean.tsv"
patho = "/Users/jiayao/Work/DeepSeq/src/hgmd_clinvar_cancer_combined_pathogenic_variants.tsv"

In [ ]:
class Gene:
    def __init__(self, gene, uniprot):
        self.uniprot = uniprot
        self.gene = gene
        self.Bmutations = []
        self.Pmutations = []
genedict = {}
P = csv.reader(open(patho, 'rb'), delimiter="\t")
P.next()
for row in P:
    uniprot, gene, mut = row[10], row[0], row[16]
    if gene not in genedict:
        genedict[gene] = Gene(gene, uniprot)
    genedict[gene].Pmutations.append(mut)
    
B = csv.reader(open(benigh, 'rb'), delimiter="\t")
B.next()
for row in B:
    uniprot, gene, mut = row[10], row[0], row[16]
    if gene not in genedict:
        genedict[gene] = Gene(gene, uniprot)
    genedict[gene].Bmutations.append(mut)

In [ ]:
Bs, Ps = [], []
Selected = []
for K, gene in genedict.items():
    Bs.append(len(gene.Bmutations))
    Ps.append(len(gene.Pmutations))
    if len(gene.Pmutations) > 43 and len(gene.Bmutations) > 43 and gene.uniprot in Gene2PDB:
        Selected.append(gene)
print len(Selected)

In [ ]:
bbs = [float(x)/10 for x in Bs]
pps = [x for x in Ps]
plt.scatter(bbs, pps)
plt.show()

In [ ]:
plt.hist(Bs, bins=1000, color="blue")
plt.hist(Ps, bins=1000, color="red")
plt.xlim(0,200)
plt.show()

In [ ]:
Bs, Ps = [], []
for gene in Selected:
    Bs.append(len(gene.Bmutations))
    Ps.append(len(gene.Pmutations))

In [ ]:
plt.scatter([float(x)/10 for x in Bs], Ps)
plt.show()

In [ ]:
fout = open("selected.genes.txt", 'wb')
geneset = []
for gene in Selected:
    geneset.append(gene.gene) 
    fout.write(gene.gene+"\n")
geneset = set(geneset)
print len(geneset)

In [ ]:
SP = csv.writer(open("Pathogenic.selected.tsv", 'wb'), delimiter="\t")
SB = csv.writer(open("Benigh.selected.tsv", 'wb'), delimiter="\t")


P = csv.reader(open(patho, 'rb'), delimiter="\t")
SP.writerow(P.next())
for row in P:
    gene, mut = row[0], row[16]
    if gene in geneset:
        SP.writerow(row)
    
B = csv.reader(open(benigh, 'rb'), delimiter="\t")
SB.writerow(B.next())
for row in B:
    gene, mut = row[0], row[16]
    if gene in geneset:
        SB.writerow(row)

In [ ]:
def AnnotatePDBFname(inpF, outF):
    Fin = csv.reader(open(inpF, 'rb'), delimiter="\t")
    Fout = csv.writer(open(outF, 'wb'), delimiter="\t")
    Fout.writerow(["PDB.Fil"] + Fin.next())
    for row in Fin:
        uniprot, gene, mut = row[10], row[0], row[16]
        if uniprot not in Gene2PDB:
            PDB = "-"
        else:
            PDB = Gene2PDB[uniprot]
        Fout.writerow([PDB]+row)

In [ ]:
AnnotatePDBFname("Pathogenic.selected.tsv", "Pathogenic.selected.PDB.tsv")

In [ ]:
AnnotatePDBFname("Benigh.selected.tsv", "Benigh.selected.PDB.tsv")

# Select PDB file

In [99]:
MapFilHead = ["PDB", "Chain", "PDB sequential residue number", "PDB_AA", "PDB_Coord", "Uniprot", "Uni_AA", "Uni_Coord"]
PDB2Uniprot = "../dat/test.map"
ins.ProcessPDB2Uniprot(PDB2Uniprot)

P60709 3byh.A
C0Z268 3byh.B
G2SES0 5efr.A
G2SI47 5efr.A
P02185 101m.A
P17900 1g13.A
P38398 1jm7.A
Q12756 4egx.A
W5ZZF5 5w9h.A
P60709	3byh.A	3	375
C0Z268	3byh.B	389	579
G2SES0	5efr.A	523	686
P38398	1jm7.A	2	103
Q12756	4egx.B	430	605
Q12756	4uxs.C	5	356
W5ZZF5	5w9p.I	18	743
W5ZZF5	5w9p.J	753	1223


G2SI47


In [101]:
Uni2PDBFil = "Uni2PDB.res.map"
ResMapFil = "../dat/PDB2SWISS.sort.tsv"
reader = csv.reader(open(Uni2PDBFil, 'rb'), delimiter="\t")
keeps = []
for row in reader:
    Uni = row[0]
    pdb, chain = row[1].split(".")
    ID = ".".join([Uni,pdb,chain])
    keeps.append(ID)
keeps = set(keeps)
reader = csv.reader(open(ResMapFil, 'rb'), delimiter="\t")
writer = csv.writer(open("../dat/PDB2SWISS.select.tsv", 'wb'), delimiter="\t")
for row in reader:
    ID = ".".join([row[5], row[0], row[1]])
    if ID in keeps:
        writer.writerow(row)


In [102]:
0.8435/2

0.42175

In [104]:
5 * 0.18

0.8999999999999999